<a href="https://colab.research.google.com/github/lblum95/AML/blob/master/task1_Jannik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AML first project

## Data ingestion

### Connect to drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Import all data libraries

In [2]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn import preprocessing
import numpy as np

### Import data

In [3]:
x_train = pd.read_csv("data/X_train.csv",index_col=0,header = 0)
y_train = pd.read_csv("data/y_train.csv",index_col=0,header = 0)
x_test = pd.read_csv("data/X_test.csv",index_col=0,header = 0)

### Data cleaning


Outlier clipping

quantile ignores nans :)

In [4]:
percentile=0.05
#quantile ignores nans :)
print(x_train.shape)
lower=x_train.quantile(percentile, axis=0, interpolation='lower')
higher=x_train.quantile(1-percentile, axis=0, interpolation='higher')
x_train=x_train.clip(lower,higher,axis=1)

(1212, 832)




Fill mean in NaNs

In [5]:
#fill NaNs with mean
x_train=x_train.apply(lambda x: x.fillna(x.mean()),axis=0)
#fill NaNs with mean
x_test=x_test.apply(lambda x: x.fillna(x.mean()),axis=0)

Normalize data

In [6]:
scaler = preprocessing.StandardScaler().fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

## Feature selection

### import all libraries

In [7]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

### Fiddle down to 200 features

In [8]:
kbest=SelectKBest(f_regression, k=200)
x_train = kbest.fit_transform(x_train, np.ravel(y_train))
print(x_train.shape)

(1212, 200)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


### RFECV with linear SVR 

In [9]:
linear = SVR(kernel="linear")
rfe_linear = RFECV(linear, step=5, cv=5)
train_linear = rfe_linear.fit_transform(x_train, np.ravel(y_train))
print(train_linear.shape)

(1212, 40)


Test SVR

In [10]:
train_x, val_x, train_y, val_y = train_test_split(train_linear, y_train, test_size=0.2, random_state=36)
linear = SVR(kernel="linear")
clf = linear.fit(train_x, np.ravel(train_y))
print(clf.score(val_x, np.ravel(val_y))) #already correct r2 metric :)

0.5074965285156073


In [11]:
test_linear=kbest.transform(x_test)
test_linear=rfe_linear.transform(test_linear)
y_test=clf.predict(test_linear)
#Write predictions
df = pd.DataFrame(y_test)
df.to_csv('data/Linear_Jannik.csv',header = ['y'], index_label = 'id')

AttributeError: ignored

In [16]:
tree = DecisionTreeRegressor(random_state=0)
rfe_tree = RFECV(tree, step=5, cv=5)
train_tree = rfe_tree.fit_transform(x_train, np.ravel(y_train))
print(train_tree.shape)

(1212, 25)


In [17]:
train_x, val_x, train_y, val_y = train_test_split(train_tree, y_train, test_size=0.2, random_state=36)
tree = DecisionTreeRegressor(random_state=0)
clf = tree.fit(train_x, np.ravel(train_y))
print(clf.score(val_x, np.ravel(val_y))) #already correct r2 metric :)

0.18383500900706728


In [18]:
test_tree=kbest.transform(x_test)
test_tree=rfe_tree.transform(test_tree)
y_test=clf.predict(test_tree)
df = pd.DataFrame(y_test)
df.to_csv('data/Tree_Jannik.csv',header = ['y'], index_label = 'id')